In [1]:
import sys
import warnings
import os
import numpy as np
import pandas as pd

In [2]:
path = "/home/srishti"

df=pd.read_csv(os.path.join(path, "Consolidated_AE_mastersheet.csv"))
df

/home/srishti/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (13,18,19,31,32,34,35,46) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,State_Name,Assembly_No,Constituency_No,Year,month,DelimID,Poll_No,Position,Candidate,Sex,...,Contested,Last_Party,Last_Party_ID,Last_Constituency_Name,Same_Constituency,Same_Party,No_Terms,Turncoat,Incumbent,Recontest
0,Andhra_Pradesh,3,1,1962.0,2,1,0,1,KIRTI CHANDRA DEO,M,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,False,False,False
1,Andhra_Pradesh,3,1,1962.0,2,1,0,2,DAKKATA PITAMBARU,M,...,1.0,NaN,NaN,NaN,NaN,NaN,0.0,False,False,False
2,Andhra_Pradesh,3,2,1962.0,2,1,0,1,GOUTHU LATCHANNA,M,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,False,False,False
3,Andhra_Pradesh,3,2,1962.0,2,1,0,2,MAJJI TULASIDASS,M,...,1.0,NaN,NaN,NaN,NaN,NaN,0.0,False,False,False
4,Andhra_Pradesh,3,2,1962.0,2,1,0,3,GANNI PADMANABHA RAO,M,...,1.0,NaN,NaN,NaN,NaN,NaN,0.0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464607,West_Bengal,17,294,2021.0,5,4,0,3,ASIF EKBAL,M,...,1.0,NaN,NaN,NaN,NaN,NaN,0.0,False,False,False
464608,West_Bengal,17,294,2021.0,5,4,0,4,ANSARUL SAIEKH,M,...,1.0,NaN,NaN,NaN,NaN,NaN,0.0,False,False,False
464609,West_Bengal,17,294,2021.0,5,4,0,5,TARAK LET,M,...,2.0,BSP,16651.0,RAMPURHAT,False,True,0.0,False,False,True
464610,West_Bengal,17,294,2021.0,5,4,0,6,None Of The Above,NaN,...,NaN,NaN,5897.0,NaN,NaN,NaN,NaN,NaN,False,False


In [3]:
df['State_Name']=df['State_Name'].mask(df['State_Name']=='Uttar_Pradesh','Uttar Pradesh')

In [4]:
states=['Goa','Manipur','Punjab','Uttar Pradesh','Uttarakhand']

df_assembly=df.groupby(['State_Name'])['Assembly_No'].max().reset_index()

df_assembly=df_assembly[df_assembly['State_Name'].isin(states)]

df_assembly

,State_Name,Assembly_No
6,Goa,7
18,Manipur,12
25,Punjab,15
31,Uttar Pradesh,17
32,Uttarakhand,4


In [5]:
df[(df['State_Name'].isin(states)) & (df['Party']!='NOTA')]['Sex'].unique()

array(['M', 'F', nan, 'O'], dtype=object)

In [6]:
len(df[(df['State_Name'].isin(states)) & (df['Party']!='NOTA')])

98103

In [56]:
len(df[(df['State_Name'].isin(states)) & (df['Party']!='NOTA') & (df['Sex'].isna()==True)])

1047

In [57]:
x=1047*100/98103

x

1.0672456499801228

In [58]:
#NOTA calculation


df_n=df[ (df['State_Name'].isin(states)) & (df['Poll_No']==0) & (df['Party']=='NOTA')]

df_indivi=df_n.groupby(['State_Name','Assembly_No','Year'])['Votes'].sum()

df_indivi=df_indivi.reset_index()

df_total=df_n.groupby(['State_Name','Assembly_No','Year'])['Valid_Votes'].sum()

df_total=df_total.reset_index()

merged_nota=df_total.merge(df_indivi,how='left',on=['State_Name','Assembly_No','Year'])

merged_nota['Percentage']=np.nan
merged_nota['Percentage']=merged_nota['Votes']*100/merged_nota['Valid_Votes']

print(merged_nota)

merged_nota.to_csv('nota.csv')

      State_Name  Assembly_No    Year  Valid_Votes     Votes  Percentage
0            Goa            7  2017.0     916216.0   10919.0    1.191750
1        Manipur           12  2017.0    1657776.0    9062.0    0.546636
2         Punjab           15  2017.0   15443460.0  108456.0    0.702278
3  Uttar Pradesh           17  2017.0   86727767.0  757643.0    0.873588
4    Uttarakhand            4  2017.0    4975167.0   50426.0    1.013554


In [59]:
#Total Voter Turnout 

df_v=df[(df['State_Name'].isin(states))]

#Valid Votes, #use of max function is to get an integer which is same otherwise, using unique gives list 
df_valid_votes=df_v.groupby(['State_Name','Assembly_No','Year','Constituency_No','Poll_No'])['Valid_Votes'].unique().reset_index().explode('Valid_Votes')
df_valid_votes=df_assembly.merge(df_valid_votes, how='left', on=['State_Name','Assembly_No'])

#To check for duplicates
print("Duplicates rows, if any",df_valid_votes[df_valid_votes.duplicated(subset=['State_Name','Assembly_No','Constituency_No'], keep=False)==True])

df_valid_votes=df_valid_votes.groupby(['State_Name','Assembly_No','Year'])['Valid_Votes'].sum().reset_index()

#Electors 

df_electors=df_v.groupby(['State_Name','Assembly_No','Year','Constituency_No','Poll_No'])['Electors'].unique().reset_index().explode('Electors')
df_electors=df_assembly.merge(df_electors, how='left', on=['State_Name','Assembly_No'] )

#To check for duplicates
print("Duplicates rows, if any",df_electors[df_electors.duplicated(subset=['State_Name','Assembly_No','Constituency_No'], keep=False)==True])

df_electors=df_electors.groupby(['State_Name','Assembly_No','Year'])['Electors'].sum().reset_index()

#Merging dataframes
merged_turnout=df_electors.merge(df_valid_votes, how='left', on=['State_Name','Assembly_No','Year'])
merged_turnout['Percentage']=np.nan
merged_turnout['Percentage']=merged_turnout['Valid_Votes']*100/merged_turnout['Electors']

print('Merged Turnout calculated\n', merged_turnout)

Duplicates rows, if any Empty DataFrame
Columns: [State_Name, Assembly_No, Year, Constituency_No, Poll_No, Valid_Votes]
Index: []
Duplicates rows, if any Empty DataFrame
Columns: [State_Name, Assembly_No, Year, Constituency_No, Poll_No, Electors]
Index: []
Merged Turnout calculated
       State_Name  Assembly_No    Year     Electors  Valid_Votes  Percentage
0            Goa            7  2017.0    1111691.0     916216.0   82.416427
1        Manipur           12  2017.0    1914543.0    1657776.0   86.588601
2         Punjab           15  2017.0   20029499.0   15443460.0   77.103576
3  Uttar Pradesh           17  2017.0  141664587.0   86727767.0   61.220499
4    Uttarakhand            4  2017.0    7606332.0    4975167.0   65.408228


In [60]:
#Voter Turnout 
df_turnout=pd.read_csv('../ae_voter_turnouts.csv')
gender_voter=df_assembly.merge(df_turnout, how='left', on=['State_Name','Assembly_No'])
gender_voter=gender_voter.merge(merged_turnout[['State_Name','Assembly_No','Year','Percentage']], how='left',on=['State_Name','Assembly_No'])

In [61]:
gender_voter.drop(['Year_x'], axis=1, inplace=True)

gender_voter.rename({'Year_y':'Year'}, axis=1, inplace=True)

gender_voter['total']=gender_voter['total'].mask(gender_voter['total'].isna(), gender_voter['Percentage'])

gender_voter.to_csv('voter.csv')

In [62]:
#Party Performance 

df_party_con=df[(df['State_Name'].isin(states))].groupby(['State_Name','Assembly_No'])['Party'].unique().reset_index()

df_party_con['Contested_Count']=np.nan
df_party_con['Contested_Count']=df_party_con['Party'].str.len()

df_party_rep=df[(df['State_Name'].isin(states)) & (df['Position']==1)].groupby(['State_Name','Assembly_No'])['Party'].unique().reset_index()

df_party_rep['Represented_Count']=np.nan
df_party_rep['Represented_Count']=df_party_rep['Party'].str.len()

df_party=df_assembly.merge(df_party_con[['State_Name','Assembly_No','Contested_Count']], how='left', on=['State_Name','Assembly_No'])

df_party=df_party.merge(df_party_rep[['State_Name','Assembly_No','Represented_Count']], how='left', on=['State_Name','Assembly_No'])

print(df_party)

df_party.to_csv('party.csv')

      State_Name  Assembly_No  Contested_Count  Represented_Count
0            Goa            7               19                  6
1        Manipur           12               19                  7
2         Punjab           15               53                  5
3  Uttar Pradesh           17              304                  9
4    Uttarakhand            4               36                  3


In [10]:
#Women Winner

df_women_winner=df[(df['Poll_No']==0) & (df['State_Name'].isin(states)) & (df['Position']==1) & (df['Sex']=='F')].groupby(['State_Name','Assembly_No'])['Constituency_No'].unique().reset_index()

df_women_winner=df_assembly.merge(df_women_winner, how='left', on=['State_Name','Assembly_No'])

df_women_winner['count']=np.nan

df_women_winner['count']=df_women_winner['Constituency_No'].str.len()

df_women_winner

,State_Name,Assembly_No,Constituency_No,count
0,Goa,7,"[12, 27]",2
1,Manipur,12,"[19, 50]",2
2,Punjab,15,"[5, 70, 77, 93, 94, 105]",6
3,Uttar Pradesh,17,"[22, 23, 31, 32, 38, 57, 64, 68, 90, 94, 140, ...",42
4,Uttarakhand,4,"[28, 36, 45, 51, 59]",5


In [11]:
#Women Contestant

df_women_contestant=df[(df['Poll_No']==0) & (df['State_Name'].isin(states)) & (df['Sex']=='F')].groupby(['State_Name','Assembly_No'])['Candidate'].count().reset_index()

df_women_contestant=df_assembly.merge(df_women_contestant, how='left', on=['State_Name','Assembly_No'])

df_women_contestant.rename({'Candidate':'Women'}, axis=1, inplace=True)

df_women_contestant

,State_Name,Assembly_No,Women
0,Goa,7,19
1,Manipur,12,11
2,Punjab,15,81
3,Uttar Pradesh,17,482
4,Uttarakhand,4,62


In [17]:
#Women Winner

df_men_winner=df[(df['Poll_No']==0) & (df['State_Name'].isin(states)) & (df['Position']==1) & (df['Sex']=='M')].groupby(['State_Name','Assembly_No'])['Constituency_No'].unique().reset_index()

df_men_winner=df_assembly.merge(df_men_winner, how='left', on=['State_Name','Assembly_No'])

df_men_winner['count']=np.nan

df_men_winner['count']=df_men_winner['Constituency_No'].str.len()

df_men_winner

,State_Name,Assembly_No,Constituency_No,count
0,Goa,7,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15...",38
1,Manipur,12,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",58
2,Punjab,15,"[1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",111
3,Uttar Pradesh,17,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",361
4,Uttarakhand,4,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",65


In [15]:
#Men Contestant

df_men_contestant=df[(df['Poll_No']==0) & (df['State_Name'].isin(states)) & (df['Sex']=='M')].groupby(['State_Name','Assembly_No'])['Candidate'].count().reset_index()

df_men_contestant=df_assembly.merge(df_men_contestant, how='left', on=['State_Name','Assembly_No'])

df_men_contestant.rename({'Candidate':'Men'}, axis=1, inplace=True)

df_men_contestant

,State_Name,Assembly_No,Men
0,Goa,7,232
1,Manipur,12,255
2,Punjab,15,1063
3,Uttar Pradesh,17,4370
4,Uttarakhand,4,573


In [13]:
#Constituencies 

df_constituencies=df[(df['State_Name'].isin(states))].groupby(['State_Name','Assembly_No','Constituency_Type'])['Constituency_No'].unique().reset_index()


df_constituencies['count']=np.nan

df_constituencies['count']=df_constituencies['Constituency_No'].str.len()

temp=df_constituencies.groupby(['State_Name','Assembly_No'])['count'].sum().reset_index()


df_constituencies=df_constituencies.merge(temp, how='left', on=['State_Name','Assembly_No'])

df_constituencies.rename({'count_x':'count_i','count_y':'total'}, axis=1, inplace=True)

df_constituencies=df_assembly.merge(df_constituencies, on=['State_Name','Assembly_No'], how='left')
df_constituencies.to_csv('constituencies.csv')

#df_constituencies=df_constituencies.drop_duplicates(subset=['State_Name','Assembly_No','total'], keep="first")

df_constituencies

,State_Name,Assembly_No,Constituency_Type,Constituency_No,count_i,total
0,Goa,7,GEN,"[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",39,40
1,Goa,7,SC,[2],1,40
2,Manipur,12,GEN,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",40,60
3,Manipur,12,SC,[16],1,60
4,Manipur,12,ST,"[41, 42, 43, 44, 45, 46, 47, 48, 49, 51, 52, 5...",19,60
5,Punjab,15,GEN,"[1, 3, 4, 6, 7, 9, 10, 11, 12, 13, 15, 17, 18,...",83,117
6,Punjab,15,SC,"[2, 5, 8, 14, 16, 20, 25, 29, 30, 33, 34, 38, ...",34,117
7,Uttar Pradesh,17,GEN,"[1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14, 15, 1...",318,403
8,Uttar Pradesh,17,SC,"[6, 13, 18, 21, 31, 38, 39, 45, 59, 70, 71, 77...",85,403
9,Uttarakhand,4,GEN,"[2, 3, 4, 6, 7, 8, 10, 11, 12, 13, 14, 16, 17,...",55,70


In [14]:
df_contestant=df[(df['State_Name'].isin(states)) & (df['Party']!='NOTA') & (df['Poll_No']==0)].groupby(['State_Name','Assembly_No'])['Candidate'].count().reset_index()

df_contestant=df_assembly.merge(df_contestant, how='left', on=['State_Name','Assembly_No'])

df_contestant.rename({'Candidate':'Total'}, axis=1, inplace=True)

df_contestant=df_contestant.merge(df_women_contestant, how='left', on=['State_Name','Assembly_No'])

df_contestant=df_contestant.merge(df_women_winner, how='left', on=['State_Name','Assembly_No'])

df_contestant.rename({'count':'Winner'}, axis=1, inplace=True)

df_contestant=df_contestant.merge(df_constituencies.drop_duplicates(subset=['State_Name','Assembly_No','total'], keep="first")[['State_Name','Assembly_No','total']], how='left', on=['State_Name','Assembly_No'])

df_contestant.drop(['Constituency_No'], inplace=True, axis=1)

df_contestant

,State_Name,Assembly_No,Total,Women,Winner,total
0,Goa,7,251,19,2,40
1,Manipur,12,266,11,2,60
2,Punjab,15,1145,81,6,117
3,Uttar Pradesh,17,4853,482,42,403
4,Uttarakhand,4,637,62,5,70


In [18]:
df_contestant_m=df[(df['State_Name'].isin(states)) & (df['Party']!='NOTA') & (df['Poll_No']==0)].groupby(['State_Name','Assembly_No'])['Candidate'].count().reset_index()

df_contestant_m=df_assembly.merge(df_contestant_m, how='left', on=['State_Name','Assembly_No'])

df_contestant_m.rename({'Candidate':'Total'}, axis=1, inplace=True)

df_contestant_m=df_contestant_m.merge(df_men_contestant, how='left', on=['State_Name','Assembly_No'])

df_contestant_m=df_contestant_m.merge(df_men_winner, how='left', on=['State_Name','Assembly_No'])

df_contestant_m.rename({'count':'Winner'}, axis=1, inplace=True)

df_contestant_m=df_contestant_m.merge(df_constituencies.drop_duplicates(subset=['State_Name','Assembly_No','total'], keep="first")[['State_Name','Assembly_No','total']], how='left', on=['State_Name','Assembly_No'])

df_contestant_m.drop(['Constituency_No'], inplace=True, axis=1)

df_contestant_m

,State_Name,Assembly_No,Total,Men,Winner,total
0,Goa,7,251,232,38,40
1,Manipur,12,266,255,58,60
2,Punjab,15,1145,1063,111,117
3,Uttar Pradesh,17,4853,4370,361,403
4,Uttarakhand,4,637,573,65,70


In [20]:
df_contestant=df_contestant_m.merge(df_contestant[['State_Name','Assembly_No','Women','Winner']], how='left', on=['State_Name','Assembly_No'])

,State_Name,Assembly_No,Total,Men,Winner_x,total,Women,Winner_y
0,Goa,7,251,232,38,40,19,2
1,Manipur,12,266,255,58,60,11,2
2,Punjab,15,1145,1063,111,117,81,6
3,Uttar Pradesh,17,4853,4370,361,403,482,42
4,Uttarakhand,4,637,573,65,70,62,5


In [21]:

df_contestant.rename({'Men':'Contestant_Men', 'Women':'Contestant_Women','Total':'Total Contestants','total':'Total Winners','Winner_x':'Men Winner','Winner_y':'Women Winner'}, axis=1, inplace=True)



,State_Name,Assembly_No,Total Contestants,Contestant_Men,Men Winner,Total Winners,Contestant_Women,Women Winner
0,Goa,7,251,232,38,40,19,2
1,Manipur,12,266,255,58,60,11,2
2,Punjab,15,1145,1063,111,117,81,6
3,Uttar Pradesh,17,4853,4370,361,403,482,42
4,Uttarakhand,4,637,573,65,70,62,5


In [23]:
df_contestant['Con_M_P']=round(df_contestant['Contestant_Men']*100/df_contestant['Total Contestants'],2)
df_contestant['Con_W_P']=round(df_contestant['Contestant_Women']*100/df_contestant['Total Contestants'],2)


df_contestant['Win_M_P']=round(df_contestant['Men Winner']*100/df_contestant['Total Winners'],2)
df_contestant['Win_W_P']=round(df_contestant['Women Winner']*100/df_contestant['Total Winners'],2)

df_contestant

,State_Name,Assembly_No,Total Contestants,Contestant_Men,Men Winner,Total Winners,Contestant_Women,Women Winner,Con_M_P,Con_W_P,Win_M_P,Win_W_P
0,Goa,7,251,232,38,40,19,2,92.43,7.57,95.00,5.00
1,Manipur,12,266,255,58,60,11,2,95.86,4.14,96.67,3.33
2,Punjab,15,1145,1063,111,117,81,6,92.84,7.07,94.87,5.13
3,Uttar Pradesh,17,4853,4370,361,403,482,42,90.05,9.93,89.58,10.42
4,Uttarakhand,4,637,573,65,70,62,5,89.95,9.73,92.86,7.14


In [50]:
df_contestant['Contestant_Men']+df_contestant['Contestant_Women']

0     251
1     266
2    1144
3    4852
4     635
dtype: int64

In [49]:
# import plotly.graph_objects as go
# states=df_contestant['State_Name']

# fig = go.Figure(data=[
#     go.Bar(name='Percentage men across all contestants', x=states, y=df_contestant['Con_M_P'], 
#            marker_color='lightblue', 
           
#            customdata=np.transpose([df_contestant['Con_M_P'],df_contestant['Contestant_Men'],df_contestant['Total Contestants']]),
           
#             hovertemplate="<br>".join([
#             "Percentage: %{customdata[0]}",
#             "Number of men contestants: %{customdata[1]}",
#             "Total number of contestants: %{customdata[2]}"])
#           ),
    
#     go.Bar(name='Percentage women across all contestants', x=states, y=df_contestant['Con_W_P'], 
#            marker_color='purple',
          
#           customdata=np.transpose([df_contestant['Con_W_P'],df_contestant['Contestant_Women'],df_contestant['Total Contestants']]),
           
#             hovertemplate="<br>".join([
#             "Percentage: %{customdata[0]}",
#             "Number of women contestants: %{customdata[1]}",
#             "Total number of contestants: %{customdata[2]}"])
#           ),
# ])


# fig.update_layout(hovermode=None, plot_bgcolor="White", legend=dict(
#     yanchor="bottom",
#     y=0.99,
#     xanchor="right",
#     x=0.99
# ), barmode='group')

# fig.update_xaxes(showline=True, linewidth=1, linecolor='grey')
# fig.update_yaxes(showline=True, linewidth=1, linecolor='grey')


In [48]:
# fig = go.Figure(data=[
#     go.Bar(name='Percentage men across all winner', x=states, y=df_contestant['Win_M_P'], 
#            marker_color='lightblue', 
           
#            customdata=np.transpose([df_contestant['Win_M_P'],df_contestant['Men Winner'],df_contestant['Total Winners']]),
           
#             hovertemplate="<br>".join([
#             "Percentage: %{customdata[0]}",
#             "Number of men winner: %{customdata[1]}",
#             "Total number of winner: %{customdata[2]}"])
#           ),
    
#     go.Bar(name='Percentage women across all winner', x=states, y=df_contestant['Win_W_P'], 
#            marker_color='purple',
          
#           customdata=np.transpose([df_contestant['Win_W_P'],df_contestant['Women Winner'],df_contestant['Total Winners']]),
           
#             hovertemplate="<br>".join([
#             "Percentage: %{customdata[0]}",
#             "Number of women winner: %{customdata[1]}",
#             "Total number of winner: %{customdata[2]}"])
#           ),
# ])


# fig.update_layout(hovermode=None, plot_bgcolor="White", legend=dict(
#     yanchor="bottom",
#     y=0.99,
#     xanchor="right",
#     x=0.99
# ), barmode='group')

# fig.update_xaxes(showline=True, linewidth=1, linecolor='grey')
# fig.update_yaxes(showline=True, linewidth=1, linecolor='grey')


In [47]:
df_contestant.to_csv('contestant.csv')

In [68]:
#Newcomers

df_new_comers=df[(df['State_Name'].isin(states)) & (df['Position']==1) & (df['Contested']==1)].groupby(['State_Name','Assembly_No'])['Constituency_No'].unique().reset_index()

df_new_comers['count']=np.nan

df_new_comers['count']=df_new_comers['Constituency_No'].str.len()

df_new_comers=df_assembly.merge(df_new_comers, how='left', on=['State_Name','Assembly_No'])

print(df_new_comers)

df_new_comers.to_csv('newcomer.csv')


      State_Name  Assembly_No  \
0            Goa            7   
1        Manipur           12   
2         Punjab           15   
3  Uttar Pradesh           17   
4    Uttarakhand            4   

                                     Constituency_No  count  
0                     [6, 7, 10, 13, 27, 28, 34, 38]      8  
1      [4, 6, 8, 10, 15, 19, 23, 33, 35, 40, 41, 43]     12  
2  [2, 3, 4, 8, 14, 16, 18, 22, 24, 25, 26, 28, 3...     58  
3  [1, 4, 5, 6, 7, 8, 9, 10, 12, 13, 15, 16, 17, ...    238  
4  [5, 6, 7, 8, 9, 10, 18, 29, 33, 36, 37, 39, 41...     20  


In [69]:
df

,State_Name,Assembly_No,Constituency_No,Year,month,DelimID,Poll_No,Position,Candidate,Sex,...,Contested,Last_Party,Last_Party_ID,Last_Constituency_Name,Same_Constituency,Same_Party,No_Terms,Turncoat,Incumbent,Recontest
0,Andhra_Pradesh,3,1,1962.0,2,1,0,1,KIRTI CHANDRA DEO,M,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,False,False,False
1,Andhra_Pradesh,3,1,1962.0,2,1,0,2,DAKKATA PITAMBARU,M,...,1.0,NaN,NaN,NaN,NaN,NaN,0.0,False,False,False
2,Andhra_Pradesh,3,2,1962.0,2,1,0,1,GOUTHU LATCHANNA,M,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,False,False,False
3,Andhra_Pradesh,3,2,1962.0,2,1,0,2,MAJJI TULASIDASS,M,...,1.0,NaN,NaN,NaN,NaN,NaN,0.0,False,False,False
4,Andhra_Pradesh,3,2,1962.0,2,1,0,3,GANNI PADMANABHA RAO,M,...,1.0,NaN,NaN,NaN,NaN,NaN,0.0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464607,West_Bengal,17,294,2021.0,5,4,0,3,ASIF EKBAL,M,...,1.0,NaN,NaN,NaN,NaN,NaN,0.0,False,False,False
464608,West_Bengal,17,294,2021.0,5,4,0,4,ANSARUL SAIEKH,M,...,1.0,NaN,NaN,NaN,NaN,NaN,0.0,False,False,False
464609,West_Bengal,17,294,2021.0,5,4,0,5,TARAK LET,M,...,2.0,BSP,16651.0,RAMPURHAT,False,True,0.0,False,False,True
464610,West_Bengal,17,294,2021.0,5,4,0,6,None Of The Above,NaN,...,NaN,NaN,5897.0,NaN,NaN,NaN,NaN,NaN,False,False


In [70]:
df_party_seat=df[(df['State_Name'].isin(states)) & (df['Position']==1) & (df['Poll_No']==0)]
df_party_seat=df_assembly.merge(df_party_seat, how='left', on=['State_Name','Assembly_No'])

df_party_seat=df_party_seat.groupby(['State_Name','Assembly_No','Party'])['Constituency_No'].count().reset_index()

df_party_seat.sort_values(by=['State_Name','Assembly_No','Constituency_No'], ascending=False, inplace=True)

df_party_seat=df_party_seat.merge(df_contestant[['State_Name','Assembly_No','total']], how='left', on=['State_Name','Assembly_No'])

df_party_seat.rename({'Constituency_No':'Party_Seat'}, axis=1, inplace=True)

df_party_seat['Party_Seat_Percentage']=np.nan

df_party_seat['Party_Seat_Percentage']=df_party_seat['Party_Seat']*100/df_party_seat['total']



In [71]:
df_party_seat['Party_Seat_Percentage']=round(df_party_seat['Party_Seat_Percentage'],2)
df_party_seat.sort_values(by=['State_Name','Party_Seat_Percentage'], inplace=True, ascending=False)

df_party_seat_k=df_party_seat.groupby(['State_Name']).apply(lambda x: x.nlargest(5, 'Party_Seat_Percentage'))
df_party_seat_k=df_party_seat_k.set_index('State_Name')
df_party_seat_k=df_party_seat_k.reset_index()
df_party_seat_x=df_party_seat_k.append(df_party_seat)

df_party_seat_other=df_party_seat_x.drop_duplicates(subset=['State_Name','Assembly_No','Party','Party_Seat','total','Party_Seat_Percentage'], keep=False)
df_party_seat_o=df_party_seat_other.groupby(['State_Name','Assembly_No','total'])['Party_Seat_Percentage'].sum().reset_index()

df_party_seat_p=df_party_seat_other.groupby(['State_Name','Assembly_No','total'])['Party_Seat'].sum().reset_index()
df_party_seat_p['Party']='Other'
df_party_seat_o=df_party_seat_o.merge(df_party_seat_p, how='left', on=['State_Name','Assembly_No','total'])
df_party_seat_k=df_party_seat_k.append(df_party_seat_o).reset_index()

In [72]:
df_party_seat_k.to_csv('party_seat_share.csv')

In [75]:
df_party_seat=pd.read_csv('./party_seat_share.csv')
df_party_seat.drop(['Unnamed: 0','index'], axis=1, inplace=True)

party_presence=df_party_seat.groupby(['Party'])['State_Name'].unique().reset_index()

party_presence['count']=party_presence['State_Name'].str.len()

party_presence=party_presence[party_presence['count']>1]

party_presence=party_presence.explode('State_Name').merge(df_party_seat, how='left', on=['Party','State_Name'])

party_presence=party_presence[party_presence['Party']!='Other']

party_presence.to_csv('party_presence.csv')